In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinu

In [ ]:
%cd '/content/drive/MyDrive/'

/content/drive/MyDrive


In [ ]:
!unzip /content/data.zip

Archive:  /content/data.zip
  inflating: data/val.csv            
  inflating: data/train.csv          
  inflating: data/test.csv           
  inflating: data/final_val.csv      
  inflating: data/final_test.csv     
  inflating: data/final_train.csv    


# *Importing libraries*

In [2]:
import json
import pickle
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")

train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences'])

# Code for Trans-Literared Text using IndicTrans Library

In [4]:
!git clone https://github.com/libindic/indic-trans.git
%cd /content/indic-trans
!pip install -r requirements.txt
!pip install .

Cloning into 'indic-trans'...
remote: Enumerating objects: 2225, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 2225 (delta 8), reused 10 (delta 3), pack-reused 2206 (from 1)
Receiving objects: 100% (2225/2225), 516.51 MiB | 14.65 MiB/s, done.
Resolving deltas: 100% (1102/1102), done.
Updating files: 100% (719/719), done.
/content/indic-trans
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.4 MB/s eta 0:00:00
Processing /content/indic-trans
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for indictrans: filename=indictrans-1.2.3-cp310-cp310-linux_x86_64.whl size=337821300 sha256=c4b91d09bed097396983c73ad76ead2c9a251d5cfd301c837a97bc5f7d05fd88
  Stored in directory: /root/.cache/pip/wheels/3e/c9/43/39c5aaa9a570043089bac219a37343294fa5b47f1350dea53a
Successfully bui

In [5]:
!indictrans --h

usage: indictrans [-h] [-v] [-s] [-t] [-b] [-m | -r] [-i] [-o]

Transliterator for Indian Languages including English

options:
  -h, --help          show this help message and exit
  -v, --version       show program's version number and exit
  -s , --source       select language (3 letter ISO-639 code) {hin, guj, pan, ben, mal, kan, tam,
                      tel, ori, eng, mar, nep, bod, kok, asm, urd}
  -t , --target       select language (3 letter ISO-639 code) {hin, guj, pan, ben, mal, kan, tam,
                      tel, ori, eng, mar, nep, bod, kok, asm, urd}
  -b, --build-lookup  build lookup to fasten transliteration
  -m, --ml            use ML system for transliteration
  -r, --rb            use rule-based system for transliteration
  -i , --input        <input-file>
  -o , --output       <output-file>


In [6]:
%cd /content/

/content


In [7]:
from indictrans import Transliterator
trn = Transliterator(source='hin', target='eng', build_lookup=True)
hin = """कांग्रेस पार्टी अध्यक्ष सोनिया गांधी, तमिलनाडु की मुख्यमंत्री
     जयललिता और रिज़र्व बैंक के गवर्नर रघुराम राजन के बीच एक समानता
     है. ये सभी अलग-अलग कारणों से भारतीय जनता पार्टी के राज्यसभा सांसद
     सुब्रमण्यम स्वामी के निशाने पर हैं. उनके जयललिता और सोनिया गांधी के
     पीछे पड़ने का कारण कथित भ्रष्टाचार है."""
eng = trn.transform(hin)
print(eng)

congress party adhyaksh sonia gandhi, tamilnadu kii mukhyamantri
     jayalalita or reserve bank ke governor raghuram rajan ke bich ek samanta
     he. ye sabhi alag-alag kaarnon se bhartiya janata party ke rajyasabha saansad
     subramanyam swami ke nishane par hai. unke jayalalita or sonia gandhi ke
     peeche padane kaa kaaran kathith bhrashtachar he.


In [8]:
trn_en_hin = Transliterator(source='eng', target='hin', build_lookup=True)

In [10]:
with open('/content/rh-code-mixed-1.pkl', 'rb') as f:
    related_words = pickle.load(f)

In [11]:
related_words

{'sarmshar': {'related_words': [{'word': 'sharmshar',
    'sequence_matcher_score': 0.9411764705882353,
    'h_script': 'शर्म्शर्',
    'tag': 'OOV'}],
  'h_script': 'सर्मशार',
  'corrected_word': 'sarma share',
  'language': 'EN',
  'check': 1},
 'laut': {'related_words': [],
  'h_script': 'लौत',
  'language': 'HI',
  'check': 1},
 'andhjihadi': {'related_words': [],
  'h_script': 'अन्ध्जिहदि',
  'corrected_word': 'andha jihaadi',
  'language': 'HI',
  'check': 1},
 'wadi': {'related_words': [], 'h_script': 'वदि', 'language': 'HI'},
 'theazad': {'related_words': [],
  'h_script': 'थेअज़द्',
  'corrected_word': 'the azad',
  'language': 'HI',
  'check': 1},
 'rahena': {'related_words': [{'word': 'rahenga',
    'sequence_matcher_score': 0.9230769230769231,
    'h_script': 'रहेन्ग',
    'tag': 'HI'}],
  'h_script': 'रहेना',
  'language': 'HI',
  'check': 1},
 'gdari': {'related_words': [], 'h_script': 'ग्दरि', 'language': 'HI'},
 'kep': {'related_words': [],
  'h_script': 'केप',
  'correc

In [ ]:
for key, value in related_words.items():
    # Use the corrected word if present, else use the key
    corrected_word = value.get('corrected_word', key)
    if corrected_word:
        # Transliterate to Devanagari script
        transliterated_word = trn_en_hin.transform(corrected_word)
        print(">>>>>>",transliterated_word)
        related_words[key]['h_script'] = transliterated_word
    else:
        corrected_word = key
        transliterated_word = trn_en_hin.transform(corrected_word)
        print("??????",corrected_word)
        related_words[key]['h_script'] = transliterated_word

In [16]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [17]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [18]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [19]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [20]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 3060
    })
    val: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 1021
    })
})

## Transliterated Word-Language Approach

In [22]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'corrected_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvaar khol di unhone isme tumhari koi galti nhi hai',
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI'}

In [30]:
related_words['hindu']['h_script']

'हिन्दू'

In [33]:
def transliterated_preprocess(sentences, related_words):
    texts = sentences['clean_text']
    processed_texts = []

    for text in texts:
        words = text.split()
        process_text = []

        for word in words:
            # Check if the word exists in the related_words dictionary
            if word in related_words:
                # Append the h_script value from related_words to the interleaved_text list
                process_text.append(related_words[word]['h_script'])
            else:
                # If the word is not in related_words, just append the original word
                process_text.append(word)

        # Join the list back into a string
        processed_texts.append(" ".join(process_text))

    return {"transliterated_text": processed_texts}

dataset = dataset.map(
    transliterated_preprocess,
    fn_kwargs={'related_words': related_words},
    batched=True
)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [34]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'corrected_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvaar khol di unhone isme tumhari koi galti nhi hai',
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'transliterated_text': 'हिन्दू तो कुछ हैं पर तुम तो पोरी दुनिया मैं गंदगी मचा रे हो और बात री भगवे की तो तुम्हे आबा के अबे भगवा ही थे तलवार के दर से सल्वार खोल दी उन्होंने इसमें तुम्हारी कोई गलती न्ही हैं'}

In [35]:
print(dataset['train'][0]['transliterated_text'])

हिन्दू तो कुछ हैं पर तुम तो पोरी दुनिया मैं गंदगी मचा रे हो और बात री भगवे की तो तुम्हे आबा के अबे भगवा ही थे तलवार के दर से सल्वार खोल दी उन्होंने इसमें तुम्हारी कोई गलती न्ही हैं


In [36]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'corrected_text': 'hindu rastra ke rujhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI',
 'transliterated_text': 'हिन्दू रास्त्र के रुझन मियाँ भाई को कस्त हैं चलो हिन्दू इस बात पे मस्त हैं'}

In [37]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'corrected_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI',
 'transliterated_text': 'जय हिन्द , इन नमुनो को बात भी करनी नहीं अति हैं , इनको पेंशन चाहिए'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [38]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
print(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

119547


In [39]:
len(tokenizer)

119547

In [42]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['transliterated_text'],max_length=97,padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","corrected_text","language_tags","transliterated_text"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [43]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  21075,  21042,  22022,  11716,  12213,    880,  14070,  13841,
          21042,    885,  41937,  10914,  83874,    889,  99007,    867,  52768,
          34315,    889,  22078,    891,  11554,  13220,  10977,  70811,    891,
          10914,    888,  19741,  72109,  10826,  21042,    880,  14070,  45753,
         103860,    852,  57381,  10412,  49545,  11554,    888,  19741,  28960,
          14080,  17798,    880,  11714,  52884,  10412, 100906,  11072,    898,
          11714,  27155,  19885,    866,  51140,  41607,  27640,  53744,    880,
          14070,  45753,  42263,  10914,  38207,    867,  11714,  18406,    884,
          99386,  11716,    102,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [44]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'हिन्दू', 'तो', 'कुछ', 'हैं', 'पर', 'त', '##ु', '##म', 'तो', 'प', '##ोर', '##ी', 'दुनिया', 'म', '##ैं', 'ग', '##ंद', '##गी', 'म', '##चा', 'र', '##े', 'हो', 'और', 'बात', 'र', '##ी', 'भ', '##ग', '##वे', 'की', 'तो', 'त', '##ु', '##म्', '##हे', 'आ', '##बा', 'के', 'अब', '##े', 'भ', '##ग', '##वा', 'ही', 'थे', 'त', '##ल', '##वार', 'के', 'दर', 'से', 'स', '##ल', '##्व', '##ार', 'ख', '##ोल', 'दी', 'उन्होंने', 'इसमें', 'त', '##ु', '##म्', '##हार', '##ी', 'कोई', 'ग', '##ल', '##ती', 'न', '##्ही', 'हैं', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [46]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = dataset['train'][0]['transliterated_text']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 हिन्दू तो कुछ हैं पर तुम तो पोरी दुनिया मैं गंदगी मचा रे हो और बात री भगवे की तो तुम्हे आबा के अबे भगवा ही थे तलवार के दर से सल्वार खोल दी उन्होंने इसमें तुम्हारी कोई गलती न्ही हैं
tokenize:              ['हिन्दू', 'तो', 'कुछ', 'हैं', 'पर', 'त', '##ु', '##म', 'तो', 'प', '##ोर', '##ी', 'दुनिया', 'म', '##ैं', 'ग', '##ंद', '##गी', 'म', '##चा', 'र', '##े', 'हो', 'और', 'बात', 'र', '##ी', 'भ', '##ग', '##वे', 'की', 'तो', 'त', '##ु', '##म्', '##हे', 'आ', '##बा', 'के', 'अब', '##े', 'भ', '##ग', '##वा', 'ही', 'थे', 'त', '##ल', '##वार', 'के', 'दर', 'से', 'स', '##ल', '##्व', '##ार', 'ख', '##ोल', 'दी', 'उन्होंने', 'इसमें', 'त', '##ु', '##म्', '##हार', '##ी', 'कोई', 'ग', '##ल', '##ती', 'न', '##्ही', 'हैं']
convert_tokens_to_ids: [21075, 21042, 22022, 11716, 12213, 880, 14070, 13841, 21042, 885, 41937, 10914, 83874, 889, 99007, 867, 52768, 34315, 889, 22078, 891, 11554, 13220, 10977, 70811, 891, 10914, 888, 19741, 72109, 10826, 21042, 880, 14070, 45753, 103860, 852, 57381, 10412

# *Calculating Weights for Labels with Imbalanced Dataset*

In [47]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [48]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [49]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [50]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [66]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)

model.resize_token_embeddings(len(tokenizer))

arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_atags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }


trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [52]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [67]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_atags/log.jsonl"))

In [68]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.690127,0.757101,0.574506,0.757101,0.653284,0.757101,0.757101,0.499354,0.430881,0.000000,0.000000,0.998708,0.861761
2,No log,0.682938,0.450539,0.711660,0.450539,0.464923,0.450539,0.450539,0.574195,0.448760,0.813765,0.417445,0.334625,0.480074
3,No log,0.611528,0.685602,0.760220,0.685602,0.706207,0.685602,0.685602,0.685129,0.640397,0.684211,0.512898,0.686047,0.767896
4,No log,0.649997,0.573947,0.759588,0.573947,0.600607,0.573947,0.573947,0.655591,0.559645,0.813765,0.480287,0.497416,0.639004
5,No log,0.647457,0.710088,0.755348,0.710088,0.725222,0.710088,0.710088,0.679226,0.651375,0.619433,0.508306,0.739018,0.794444
6,No log,0.688643,0.625857,0.756865,0.625857,0.652602,0.625857,0.625857,0.669154,0.598392,0.753036,0.493369,0.585271,0.703416
7,No log,0.711063,0.704212,0.758255,0.704212,0.721163,0.704212,0.704212,0.683619,0.650263,0.643725,0.512903,0.723514,0.787623


TrainOutput(global_step=336, training_loss=0.5779561542329335, metrics={'train_runtime': 656.1905, 'train_samples_per_second': 46.633, 'train_steps_per_second': 0.731, 'train_loss': 0.5779561542329335, 'epoch': 7.0})

In [69]:
mbert_results = trainer.evaluate()

In [70]:
print(mbert_results)

{'eval_loss': 0.6115280389785767, 'eval_accuracy': 0.6856023506366308, 'eval_precision_weighted': 0.760220329323154, 'eval_recall_weighted': 0.6856023506366308, 'eval_f1_weighted': 0.7062069517012316, 'eval_recall_micro': 0.6856023506366308, 'eval_f1_micro': 0.6856023506366308, 'eval_recall_macro': 0.6851285189718482, 'eval_f1_macro': 0.6403971046645973, 'eval_recall_positive': 0.6842105263157895, 'eval_f1_positive': 0.5128983308042488, 'eval_recall_negative': 0.686046511627907, 'eval_f1_negative': 0.7678958785249457, 'eval_runtime': 5.2625, 'eval_samples_per_second': 194.013, 'eval_steps_per_second': 12.161, 'epoch': 7.0}


# **Hing-MBERT Test results**

In [71]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [72]:
test_results_mbert

PredictionOutput(predictions=array([[ 0.39804414, -0.02051465],
       [-0.05672649,  0.47537768],
       [ 0.60974866, -0.12340879],
       ...,
       [ 0.28405955,  0.1809    ],
       [-0.62316096,  0.7490577 ],
       [ 0.28965604,  0.08105152]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.6232540011405945, 'test_accuracy': 0.6699314397649363, 'test_precision_weighted': 0.7465883167892985, 'test_recall_weighted': 0.6699314397649363, 'test_f1_weighted': 0.691669860630489, 'test_recall_micro': 0.6699314397649363, 'test_f1_micro': 0.6699314397649363, 'test_recall_macro': 0.6651445249976462, 'test_f1_macro': 0.6230701732284155, 'test_recall_positive': 0.6558704453441295, 'test_f1_positive': 0.4901664145234494, 'test_recall_negative': 0.6744186046511628, 'test_f1_negative': 0.7559739319333816, 'test_runtime': 5.3986, 'test_samples_per_second': 189.125, 'test_steps_per_second': 11.855})

In [73]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 1, 0, ..., 0, 1, 0])

In [74]:
test_df['trans_predicted_labels_5k_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [75]:
print(test_results_mbert.metrics)

{'test_loss': 0.6232540011405945, 'test_accuracy': 0.6699314397649363, 'test_precision_weighted': 0.7465883167892985, 'test_recall_weighted': 0.6699314397649363, 'test_f1_weighted': 0.691669860630489, 'test_recall_micro': 0.6699314397649363, 'test_f1_micro': 0.6699314397649363, 'test_recall_macro': 0.6651445249976462, 'test_f1_macro': 0.6230701732284155, 'test_recall_positive': 0.6558704453441295, 'test_f1_positive': 0.4901664145234494, 'test_recall_negative': 0.6744186046511628, 'test_f1_negative': 0.7559739319333816, 'test_runtime': 5.3986, 'test_samples_per_second': 189.125, 'test_steps_per_second': 11.855}


# **Hing BERT Training**

In [76]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [77]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")

bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['transliterated_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","corrected_text","language_tags","transliterated_text"])
bert_dataset.set_format("torch")

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [78]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

bert_model.resize_token_embeddings(len(bert_tokenizer))
arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_atags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_atags/log.jsonl"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [79]:
# train the model
bert_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.691081,0.734574,0.612921,0.734574,0.651545,0.734574,0.734574,0.492766,0.444169,0.024291,0.042403,0.961240,0.845935
2,No log,0.685522,0.433888,0.737866,0.433888,0.436420,0.433888,0.433888,0.585266,0.433844,0.878543,0.428854,0.291990,0.438835
3,No log,0.634579,0.699314,0.734175,0.699314,0.712423,0.699314,0.699314,0.647310,0.629191,0.546559,0.467938,0.748062,0.790444
4,No log,0.608212,0.641528,0.749469,0.641528,0.667093,0.641528,0.641528,0.664329,0.606404,0.708502,0.488827,0.620155,0.723982
5,No log,0.654057,0.629775,0.745019,0.629775,0.656310,0.629775,0.629775,0.656577,0.596549,0.708502,0.480769,0.604651,0.712329
6,No log,0.581611,0.717924,0.771935,0.717924,0.734234,0.717924,0.717924,0.703690,0.667080,0.676113,0.536977,0.731266,0.797183
7,No log,0.599646,0.683643,0.778953,0.683643,0.705839,0.683643,0.683643,0.708646,0.648218,0.757085,0.536585,0.660207,0.759851
8,No log,0.593824,0.714006,0.765459,0.714006,0.730093,0.714006,0.714006,0.694214,0.660603,0.655870,0.525974,0.732558,0.795231
9,No log,0.646172,0.773751,0.762464,0.773751,0.766835,0.773751,0.773751,0.664705,0.673375,0.453441,0.492308,0.875969,0.854442
10,No log,0.686225,0.643487,0.764396,0.643487,0.669068,0.643487,0.643487,0.682160,0.613817,0.757085,0.506775,0.607235,0.720859


TrainOutput(global_step=480, training_loss=0.5971601486206055, metrics={'train_runtime': 768.8199, 'train_samples_per_second': 39.801, 'train_steps_per_second': 0.624, 'train_loss': 0.5971601486206055, 'epoch': 10.0})

In [80]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.5816107392311096, 'eval_accuracy': 0.7179236043095005, 'eval_precision_weighted': 0.7719352988682457, 'eval_recall_weighted': 0.7179236043095005, 'eval_f1_weighted': 0.7342342397887645, 'eval_recall_micro': 0.7179236043095005, 'eval_f1_micro': 0.7179236043095006, 'eval_recall_macro': 0.7036897550973439, 'eval_f1_macro': 0.667080295276482, 'eval_recall_positive': 0.6761133603238867, 'eval_f1_positive': 0.5369774919614149, 'eval_recall_negative': 0.7312661498708011, 'eval_f1_negative': 0.7971830985915492, 'eval_runtime': 5.3174, 'eval_samples_per_second': 192.011, 'eval_steps_per_second': 12.036, 'epoch': 10.0}


# **Test results for Hing-BERT**

In [81]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['trans_predicted_labels_5k_hingBert'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.6052765846252441, 'test_accuracy': 0.6875612144955926, 'test_precision_weighted': 0.7509863391752594, 'test_recall_weighted': 0.6875612144955926, 'test_f1_weighted': 0.7067309643680687, 'test_recall_micro': 0.6875612144955926, 'test_f1_micro': 0.6875612144955926, 'test_recall_macro': 0.6726375419765873, 'test_f1_macro': 0.6360844036492233, 'test_recall_positive': 0.6437246963562753, 'test_f1_positive': 0.499215070643642, 'test_recall_negative': 0.7015503875968992, 'test_f1_negative': 0.7729537366548044, 'test_runtime': 5.4644, 'test_samples_per_second': 186.845, 'test_steps_per_second': 11.712}


# **Muril cased training**

In [103]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [100]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
print(len(muril_tokenizer))

## Adding special tokens

print(len(muril_tokenizer))
muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['transliterated_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)

muril_dataset = muril_dataset.remove_columns(["clean_text","corrected_text","language_tags","transliterated_text"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)

muril_model.resize_token_embeddings(len(muril_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_atags",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=3e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_atags/log.jsonl"))

197285
197285


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [104]:
muril_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.692813,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
2,No log,0.664088,0.765916,0.772916,0.765916,0.769079,0.765916,0.765916,0.695373,0.689703,0.558704,0.535922,0.832041,0.843484
3,No log,0.620376,0.779628,0.772429,0.779628,0.775506,0.779628,0.779628,0.682364,0.688609,0.493927,0.520256,0.870801,0.856961
4,No log,0.619517,0.776690,0.769094,0.776690,0.772331,0.776690,0.776690,0.677670,0.683983,0.485830,0.512821,0.869509,0.855146
5,No log,0.623205,0.778648,0.772795,0.778648,0.775390,0.778648,0.778648,0.684475,0.689537,0.502024,0.523207,0.866925,0.855867
6,0.616800,0.604165,0.784525,0.775219,0.784525,0.778877,0.784525,0.784525,0.682837,0.691337,0.485830,0.521739,0.879845,0.860936
7,0.616800,0.632555,0.798237,0.783108,0.798237,0.785947,0.798237,0.798237,0.676720,0.693417,0.441296,0.514151,0.912145,0.872682
8,0.616800,0.646592,0.799216,0.790557,0.799216,0.793781,0.799216,0.799216,0.702175,0.711938,0.514170,0.553377,0.890181,0.870499
9,0.616800,0.726339,0.763957,0.765596,0.763957,0.764757,0.763957,0.763957,0.681676,0.680422,0.522267,0.517034,0.841085,0.843811
10,0.616800,0.789301,0.782566,0.769734,0.782566,0.774135,0.782566,0.782566,0.670519,0.681572,0.453441,0.502242,0.887597,0.860902


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=960, training_loss=0.4994548161824544, metrics={'train_runtime': 1238.736, 'train_samples_per_second': 24.703, 'train_steps_per_second': 0.775, 'train_loss': 0.4994548161824544, 'epoch': 10.0})

In [105]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.6041654944419861, 'eval_accuracy': 0.7845249755142018, 'eval_precision_weighted': 0.7752193819621992, 'eval_recall_weighted': 0.7845249755142018, 'eval_f1_weighted': 0.7788772392735568, 'eval_recall_micro': 0.7845249755142018, 'eval_f1_micro': 0.7845249755142018, 'eval_recall_macro': 0.68283746037724, 'eval_f1_macro': 0.6913373275435607, 'eval_recall_positive': 0.48582995951417, 'eval_f1_positive': 0.5217391304347826, 'eval_recall_negative': 0.8798449612403101, 'eval_f1_negative': 0.8609355246523388, 'eval_runtime': 5.2919, 'eval_samples_per_second': 192.938, 'eval_steps_per_second': 12.094, 'epoch': 10.0}


In [106]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['trans_predicted_labels_5k_muril'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

{'test_loss': 0.5970745086669922, 'test_accuracy': 0.7835455435847208, 'test_precision_weighted': 0.7809142367474194, 'test_recall_weighted': 0.7835455435847208, 'test_f1_weighted': 0.7821627941819607, 'test_recall_micro': 0.7835455435847208, 'test_f1_micro': 0.7835455435847208, 'test_recall_macro': 0.6987310255364112, 'test_f1_macro': 0.701195134709228, 'test_recall_positive': 0.5344129554655871, 'test_f1_positive': 0.5443298969072166, 'test_recall_negative': 0.8630490956072352, 'test_f1_negative': 0.8580603725112395, 'test_runtime': 5.4046, 'test_samples_per_second': 188.912, 'test_steps_per_second': 11.842}


## XLMR training

In [91]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [88]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['transliterated_text'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","corrected_text","language_tags","transliterated_text"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)


arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_atags",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_atags/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [89]:
import torch
torch.cuda.empty_cache()

In [92]:
xlmr_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.687862,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
2,No log,0.596255,0.724780,0.774991,0.724780,0.740186,0.724780,0.724780,0.708212,0.673086,0.676113,0.543089,0.740310,0.803083
3,0.640900,0.592139,0.760039,0.774390,0.760039,0.765970,0.760039,0.760039,0.701145,0.689741,0.587045,0.542056,0.815245,0.837425
4,0.640900,0.583243,0.750245,0.768844,0.750245,0.757676,0.750245,0.750245,0.694685,0.680885,0.587045,0.532110,0.802326,0.829659
5,0.640900,0.764795,0.715965,0.778892,0.715965,0.733573,0.715965,0.715965,0.713424,0.670014,0.708502,0.546875,0.718346,0.793153
6,0.457000,0.829596,0.741430,0.772257,0.741430,0.752403,0.741430,0.741430,0.702654,0.680117,0.627530,0.540070,0.777778,0.820163


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1152, training_loss=0.5169726411501566, metrics={'train_runtime': 1179.2509, 'train_samples_per_second': 25.949, 'train_steps_per_second': 1.628, 'train_loss': 0.5169726411501566, 'epoch': 6.0})

In [93]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.5832428932189941, 'eval_accuracy': 0.7502448579823702, 'eval_precision_weighted': 0.7688438227340855, 'eval_recall_weighted': 0.7502448579823702, 'eval_f1_weighted': 0.7576763029243888, 'eval_recall_micro': 0.7502448579823702, 'eval_f1_micro': 0.7502448579823702, 'eval_recall_macro': 0.6946850579041521, 'eval_f1_macro': 0.680884705190197, 'eval_recall_positive': 0.5870445344129555, 'eval_f1_positive': 0.5321100917431193, 'eval_recall_negative': 0.8023255813953488, 'eval_f1_negative': 0.8296593186372745, 'eval_runtime': 10.8925, 'eval_samples_per_second': 93.734, 'eval_steps_per_second': 5.876, 'epoch': 6.0}


In [94]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['trans_predicted_labels_5k_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

{'test_loss': 0.5565178990364075, 'test_accuracy': 0.7561214495592556, 'test_precision_weighted': 0.783118583393529, 'test_recall_weighted': 0.7561214495592556, 'test_f1_weighted': 0.7657859016785958, 'test_recall_micro': 0.7561214495592556, 'test_f1_micro': 0.7561214495592556, 'test_recall_macro': 0.7178571802194813, 'test_f1_macro': 0.6960165007622635, 'test_recall_positive': 0.6437246963562753, 'test_f1_positive': 0.5608465608465609, 'test_recall_negative': 0.7919896640826873, 'test_f1_negative': 0.8311864406779661, 'test_runtime': 11.1017, 'test_samples_per_second': 91.968, 'test_steps_per_second': 5.765}


In [107]:
test_df.to_csv('final_test_5k_transliteration_v2.csv',index=False, encoding='utf-8')